<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.


## Подготовка

In [1]:
!pip install imbalanced-learn 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer 

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline

from catboost import CatBoostClassifier

import lightgbm as ltb

import re

from imblearn.over_sampling import RandomOverSampler

import warnings
warnings.filterwarnings("ignore")

RANDOM_STATE=42

     |████████████████████████████████| 257 kB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 12.2 MB 30.6 MB/s eta 0:00:01
     |████████████████████████████████| 302 kB 95.0 MB/s eta 0:00:01
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.1
    Uninstalling scikit-learn-0.24.1:
      Successfully uninstalled scikit-learn-0.24.1


In [23]:
df = pd.read_csv('/datasets/toxic_comments.csv', index_col = 0)
df.head(10)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


In [24]:
# в свзяи с проблемой нехватки памяти, ограничим размер нашего датасета выборкой из 100000 значений
df = df.sample(100000)

In [25]:
# проверим данные на пропуски
df.isna().sum()

text     0
toxic    0
dtype: int64

In [26]:
df['toxic'].value_counts()

0    89837
1    10163
Name: toxic, dtype: int64

In [1]:
# на лицо дисбаланс классов, будем иметь это ввиду 

In [6]:
nltk.download('stopwords') 
stop_words = set(nltk_stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
# напишем функцию для лемматизации и очистки текста
def lemmatize(text):
    text = nltk.word_tokenize(text)
    text = [word for word in text if word not in stop_words] 
    text = ' '.join([lemmatizer.lemmatize(w) for w in text])
    return text

def clear_text(text):
    re_text = re.sub(r'[^a-zA-Z ]', ' ', text)
    res = " ".join(re_text.split())
    return res

In [27]:
df['lemm_text'] = df['text'].apply(clear_text)
df['lemm_text'] = df['lemm_text'].apply(lemmatize)

In [28]:
# проверим наличие дубликатов после лемматизации и очистки.
df['lemm_text'].duplicated().sum()

692

In [29]:
# удалим выявленные дубликаты
df['lemm_text'] = df['lemm_text'].drop_duplicates()

In [40]:
df = df[~df['lemm_text'].isna()]

In [41]:
print(f'До: {df.iloc[0,0]}')
print()
print(f'После: {df.iloc[0,2]}')

До: Not just church 

I think there should be something about the whole village, not just the church.

После: Not church I think something whole village church


In [42]:
# разделим данные на тренировочную и тестовую выборки
X = df.drop(['toxic', 'text'], axis=1)
y = df['toxic']
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2,
                                                    random_state=RANDOM_STATE, stratify=y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((79446, 1), (19862, 1), (79446,), (19862,))

## Обучение

In [43]:
# найдём метрику accuracy для константной модели. Будем предсказывать все твиты toxic=0

base_predicts = pd.Series(data=np.zeros((len(y_test))), index=y_test.index, dtype='int16')
base_accuacy = accuracy_score(y_test, base_predicts)
print(f'Accuracy константной модели {base_accuacy:.3f}')

Accuracy константной модели 0.898


In [44]:
# напишем функция для инициализации/обучения модели, векторизации данных и ресэмплирования классов
from imblearn.pipeline import Pipeline
def teach(model, params):
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=10000, min_df = 1)),
        ('sampler', RandomOverSampler(random_state=RANDOM_STATE)),
        ('model', model)])
    rs = RandomizedSearchCV(pipeline, params, cv=3, n_jobs=-1, scoring='f1', verbose=False)
    rs.fit(X_train['lemm_text'], y_train)
    print('Лучший результат:', rs.best_score_)
    print('Лучшие параметры:', rs.best_params_)
    return rs

In [45]:
lr_model = teach(LogisticRegression(), {'model__C':[0.1, 1.0, 10.0],
                                      'model__penalty':['l1','l2']})

Лучший результат: 0.7291655720887927
Лучшие параметры: {'model__penalty': 'l2', 'model__C': 10.0}


In [46]:
dtc_model = teach(DecisionTreeClassifier(), {'model__criterion':['gini','entropy'],
                                             'model__max_depth':range(2,10)}) 

Лучший результат: 0.5250789075653183
Лучшие параметры: {'model__max_depth': 9, 'model__criterion': 'gini'}


In [47]:
lgmb_model = teach(ltb.LGBMClassifier(random_state=RANDOM_STATE, verbose=0, force_col_wise=True),
                   {'model__learning_rate':[0.1,0.3,0.5]})
                    #'model__max_features':[None, 10000, 20000]})

Лучший результат: 0.7470179014122725
Лучшие параметры: {'model__learning_rate': 0.5}


In [48]:
cat_model = teach(CatBoostClassifier(), {'model__depth': [4,10],
                                         'model__learning_rate' : [0.01,0.03],
                                         'model__iterations' : [10, 40]})

0:	learn: 0.6887512	total: 824ms	remaining: 7.41s
1:	learn: 0.6845357	total: 1.5s	remaining: 5.99s
2:	learn: 0.6804063	total: 2.11s	remaining: 4.93s
3:	learn: 0.6763843	total: 2.71s	remaining: 4.06s
4:	learn: 0.6724859	total: 3.31s	remaining: 3.31s
5:	learn: 0.6693835	total: 3.94s	remaining: 2.63s
6:	learn: 0.6656971	total: 4.58s	remaining: 1.96s
7:	learn: 0.6624629	total: 5.21s	remaining: 1.3s
8:	learn: 0.6591535	total: 5.85s	remaining: 650ms
9:	learn: 0.6559546	total: 6.49s	remaining: 0us
0:	learn: 0.6885551	total: 719ms	remaining: 6.47s
1:	learn: 0.6841896	total: 1.32s	remaining: 5.28s
2:	learn: 0.6799855	total: 1.94s	remaining: 4.52s
3:	learn: 0.6762392	total: 2.59s	remaining: 3.88s
4:	learn: 0.6723257	total: 3.21s	remaining: 3.21s
5:	learn: 0.6689777	total: 3.83s	remaining: 2.55s
6:	learn: 0.6655735	total: 4.45s	remaining: 1.91s
7:	learn: 0.6619928	total: 5.13s	remaining: 1.28s
8:	learn: 0.6586606	total: 5.76s	remaining: 640ms
9:	learn: 0.6552811	total: 6.45s	remaining: 0us
0:	lea

На тренировочной выборке лучший результат у LGBMClassifier, поэтому протестируем её на тестовой выборке

In [49]:
res = f1_score(y_test, lgmb_model.predict(X_test['lemm_text']))
print("Результат LGBMClassifier", res.round(2))

Результат LGBMClassifier 0.75


## Выводы

Самый высокий показатель F1 у LGBMClassifier! С вероятностью 0.75 он идентифицирует токсичные комментарии, которые можно отправлять на модерацию!  